# git

In [ ]:
import ipywidgets as W, traitlets as T, pathlib as P, git as G, yaml, json, ipywidgets.embed as E
from tornado.ioloop import IOLoop
from tornado.concurrent import run_on_executor
from concurrent.futures import ThreadPoolExecutor
from wxyz.dvcs import Watcher
from pathlib import Path
import importnb

In [ ]:
with importnb.Notebook():
    from wxyz.notebooks.Design.DVCS.Trackers import _Tracker

## BaseRepo
A base representation of a file-based repo. This would live in its own file, and not know anything about a specific DVCS. 

In [ ]:
class _Repo(W.Widget):
    working_dir = T.Instance(P.Path)
    watching = T.Bool(default_value=False)
    dirty = T.Bool(default_value=False)
    changes = T.Tuple(allow_none=True)
    head = T.Unicode()
    head_history = T.Tuple()
    heads = W.trait_types.TypedTuple(T.Unicode(), default_value=tuple())
    _watcher = T.Instance(Watcher, allow_none=True)
    _trackers = W.trait_types.TypedTuple(T.Instance(_Tracker), default_value=tuple())
    _change_link = None
    
    def __init__(self, working_dir, *args, **kwargs):
        kwargs["working_dir"] = P.Path(kwargs.get("working_dir", working_dir))
        super().__init__(*args, **kwargs)
    
    @T.default("changes")
    def _default_changes(self):
        return tuple()
    
    @T.observe("watching", "path")
    def _on_watching(self, change):
        if self._watcher:
            if self._change_link:
                self._change_link.unlink()
                self._change_link = None
            self._watcher.watching = False
            self._watcher = None
        if self.watching:
            self._watcher = Watcher(self.working_dir)
            self._change_link = T.dlink((self._watcher, "changes"), (self, "changes"), self._on_watch_changes)
            self._watcher.watching = True
    
    def _on_watch_changes(self, changes):
        """ react to changes from the watcher
        """
        self.log.warn("changes %s", changes)
        return changes
    
    def track(self, **kwargs):
        """ create a tracker for the given widget
        """
        assert self.working_dir.exists()
        kwargs["path"] = P.Path(self.working_dir / kwargs["path"])
        tracker_cls = kwargs.pop("tracker_cls", None)
        if tracker_cls is None:
            tracker_cls = _Tracker._detect_tracker(kwargs["path"])
        assert tracker_cls is not None
        tracker = tracker_cls(**kwargs)
        self._trackers += tracker,
        return tracker

## git

Specialization for supporting `git`.

In [ ]:
class Repo(_Repo):
    _git = T.Instance(G.Repo, allow_none=True)
    
    @T.observe("working_dir")
    def _on_path(self, change):
        if change.new:
            self._git = G.Repo.init(change.new)
            ignore = Path(change.new) / ".gitignore"
            if not ignore.exists():
                ignore.write_text(".ipynb_checkpoints/")
                self.commit("initial commit")
            else:
                self._update_head_history()
    
    @T.default("head")
    def _default_head(self):
        return self._git.head.name
    
    @T.observe("head")
    def _on_head_changed(self, change):
        if change.new:
            self._update_head_history()
    
    def _update_head_history(self):
        try:
            head = [h for h in self._git.heads if h.name == self.head][0]
            self.head_history = [
                {
                    "commit": c.newhexsha, 
                    "timestamp": c.time[0],
                    "message": c.message,
                    "author": {
                        "name": c.actor.name,
                        "email": c.actor.email
                    }
                }
                for c in head.log()[::-1]
            ]
        except Exception as err:
            self.log.warn(err)
            self.head_history = []
    
    def _update_heads(self):
        """ TODO: when does this get updated?
        """
        self.heads = [head.name for head in self._git.heads]
        if self.head in [None, "HEAD"] and self.heads:
            self.head = "master"

        self._update_head_history()

    def _on_watch_changes(self, *changes):
        self.dirty = self._git.is_dirty()
        if self._watcher:
            for change in self._watcher.changes:
                for tracker in self._trackers:
                    tracked_path = Path(self._git.working_dir) / change["path"]
                    if tracker.path.resolve() == tracked_path.resolve():
                        tracker._on_file_change(None)
        return [
            dict(
                a_path=diff.a_path,
                b_path=diff.b_path,
                change_type=diff.change_type
            )
            for diff in self._git.index.diff(None)
        ] + [
            dict(
                a_path=None,
                b_path=ut,
                change_type="U"
            ) for ut in self._git.untracked_files
        ]
    
    def stage(self, path):
        self._git.index.add(path)
    
    def unstage(self, path):
        self._git.index.remove(path)
    
    def commit(self, message):
        self._git.index.commit(message)
        self._on_watching(None)
        self._update_heads()
        
    def revert(self, ref):
        self._git.head.commit = ref
        self._git.head.reset(index=True, working_tree=True)
        self._update_heads()